In [1]:
import surprise
from surprise import Dataset, Reader
from surprise import KNNBasic, KNNWithMeans, KNNBaseline
from surprise import model_selection, print_perf

In [2]:
# 加载数据
data = Dataset.load_builtin('ml-100k')
# 数据进行交叉验证
data.split(n_folds=3)

# KNNBasic

## UserCF
普通的协同过滤算法；核心思想：<br/>
1. 待预测的用户对物品i的评分，受该用户的相似用户对物品i的评分的影响<br/>
2. 从而认为待预测的用户对物品i的评分其实就是相似用户对物品i评分的加权和<br/>

简单理解：根据用户历史行为信息，将相似用户喜好的物品推荐给当前用户<br/>

流程：<br/>
1. 计算所有用户之间的相似度(一般情况下在UserCF中使用pearson相似度比较多)，从而得到相似度矩阵<br/>
2. 计算用户-物品评分矩阵; 首先从相似度矩阵中获取用户u的相似用户列表，然后根据这个列表计算用户u对物品i的评分(加权计算的方式)；迭代计算所有用户和所有物品的评分

## ItemCF
简单理解：根据用户对物品的评分信息，计算物品与物品之间的相似度；根据用户的浏览行为/评分行为/喜欢的物品，将喜欢物品的相似物品推荐给用户。


#### UserCF

In [4]:
# 模型构建
sim_options = {'name':'pearson', 'user_based': True}
algo = KNNBaseline(sim_options=sim_options)

# 模型的训练
pref = model_selection.cross_validate(algo, data, measures=['rmse', 'mae', 'fcp'], cv=3, verbose=True)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE, FCP of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9423  0.9410  0.9374  0.9402  0.0020  
MAE (testset)     0.7387  0.7402  0.7377  0.7389  0.0010  
FCP (testset)     0.7106  0.7112  0.7133  0.7117  0.0012  
Fit time          3.88    4.50    4.14    4.17    0.25    
Test time         12.55   12.69   12.73   12.66   0.08    


In [6]:
# 模型参数的给定
sim_options = {
    'name':'pearson', # 使用什么相似度度量方式
    'user_based': True # 是UserCF还是ItemCF《当参数user_based为true的时候，表示是UserCF》
}
# 构建模型
algo = KNNBasic(sim_options=sim_options)
# 进行模型训练的时候，必须对训练数据进行构建（必须构建, 其实就是将Movieline格式数据转换为稀疏矩阵，在这个转换过程中会对用户id和物品id进行重新的赋值）
# 在这个过程中，会做一个id的转换，就是将外部的数据id转换为内部数据id
trainset = data.build_full_trainset()
# 模型训练
algo.fit(trainset)

#### 计算推荐列表
row_all_user_ids = ['1','3','5','7','9'] # 所有的用户id
row_all_item_ids = ['2','4','6','8','10'] # 所有的物品id
for user_id in row_all_user_ids:
    for item_id in row_all_item_ids:
        print("用户%s对于物品%s的评分为:%.3f" % (user_id, item_id, algo.predict(user_id, item_id).est) )

Computing the pearson similarity matrix...
Done computing similarity matrix.
用户1对于物品2的评分为:3.056
用户1对于物品4的评分为:3.674
用户1对于物品6的评分为:3.678
用户1对于物品8的评分为:3.355
用户1对于物品10的评分为:3.500
用户3对于物品2的评分为:3.237
用户3对于物品4的评分为:3.417
用户3对于物品6的评分为:3.380
用户3对于物品8的评分为:4.035
用户3对于物品10的评分为:3.689
用户5对于物品2的评分为:2.936
用户5对于物品4的评分为:3.751
用户5对于物品6的评分为:3.837
用户5对于物品8的评分为:3.759
用户5对于物品10的评分为:3.646
用户7对于物品2的评分为:3.202
用户7对于物品4的评分为:4.043
用户7对于物品6的评分为:3.361
用户7对于物品8的评分为:4.389
用户7对于物品10的评分为:3.472
用户9对于物品2的评分为:3.361
用户9对于物品4的评分为:3.525
用户9对于物品6的评分为:4.040
用户9对于物品8的评分为:4.011
用户9对于物品10的评分为:3.908


In [7]:
# 获取用户u对于物品i的评分
row_user_id = '1' # 对应文件中的用户id
row_item_id = '10' # 对应文件中的电影id
inner_user_id = algo.trainset.to_inner_uid(ruid=row_user_id) # 获取内部id转换后的用户id
inner_item_id = algo.trainset.to_inner_iid(riid=row_item_id)
print("1对应的内部用户id为：", end='') 
print(inner_user_id)
print("10对应的内部物品id为:%d" % inner_item_id)
print("原始公式对应的预测评分值:", end='')
print(algo.estimate(inner_user_id, inner_item_id))
print("获取处理之后的预测评分值(数据比较准，一般使用该结果)：", end='')
print(algo.predict(row_user_id, row_item_id))

1对应的内部用户id为：455
10对应的内部物品id为:502
原始公式对应的预测评分值:(3.5000685354734964, {'actual_k': 40})
获取处理之后的预测评分值(数据比较准，一般使用该结果)：user: 1          item: 10         r_ui = None   est = 3.50   {'actual_k': 40, 'was_impossible': False}


In [8]:
# 获取近邻
row_user_id = '1' # 对应文件中的用户id
inner_user_id = algo.trainset.to_inner_uid(ruid=row_user_id) # 获取内部id转换后的用户id
user_neighbors = algo.get_neighbors(inner_user_id, k=10)
print("全部数据中最相似的k个用户:")
print(user_neighbors)

全部数据中最相似的k个用户:
[712, 714, 736, 756, 773, 789, 828, 861, 866, 924]


#### ItemCF

In [9]:
# 模型参数的给定
sim_options = {
    'name':'pearson', # 使用什么相似度度量方式
    'user_based': False # 是UserCF还是ItemCF《当参数user_based为true的时候，表示是UserCF》
}
# 构建模型
algo = KNNBasic(sim_options=sim_options, k=40)
# 对训练数据进行构建（必须构建, 其实就是将Movieline格式数据转换为稀疏矩阵，在这个转换过程中会对用户id和物品id进行重新的赋值）
trainset = data.build_full_trainset()
# 模型训练
algo.fit(trainset)

#### 计算推荐列表
row_all_user_ids = ['1','3','5','7','9'] # 所有的用户id
row_all_item_ids = ['2','4','6','8','10'] # 所有的物品id
for user_id in row_all_user_ids:
    for item_id in row_all_item_ids:
        print("用户%s对于物品%s的评分为:%.3f" % (user_id, item_id, algo.predict(user_id, item_id).est) )

Computing the pearson similarity matrix...
Done computing similarity matrix.
用户1对于物品2的评分为:2.993
用户1对于物品4的评分为:3.166
用户1对于物品6的评分为:4.007
用户1对于物品8的评分为:2.206
用户1对于物品10的评分为:2.706
用户3对于物品2的评分为:2.625
用户3对于物品4的评分为:2.298
用户3对于物品6的评分为:2.745
用户3对于物品8的评分为:2.780
用户3对于物品10的评分为:2.748
用户5对于物品2的评分为:2.399
用户5对于物品4的评分为:2.929
用户5对于物品6的评分为:2.905
用户5对于物品8的评分为:2.619
用户5对于物品10的评分为:2.471
用户7对于物品2的评分为:3.929
用户7对于物品4的评分为:4.449
用户7对于物品6的评分为:3.946
用户7对于物品8的评分为:4.092
用户7对于物品10的评分为:3.809
用户9对于物品2的评分为:4.343
用户9对于物品4的评分为:4.449
用户9对于物品6的评分为:4.576
用户9对于物品8的评分为:4.125
用户9对于物品10的评分为:4.507


In [10]:
# 直接获取相似的物品
row_item_id = '1' # 对应文件中的物品id
inner_item_id = algo.trainset.to_inner_iid(riid=row_item_id)
item_neighbors = algo.get_neighbors(inner_user_id, k=10)
print("全部数据中最相似的k个商品(内部id):{}".format(item_neighbors))
print("全部数据中最相似的k个商品(外部id):{}".format([algo.trainset.to_raw_iid(iid) for iid in item_neighbors]))
print()

# 获取这些相似用户的这个相似度
for item in item_neighbors:
    row_like_item_id = algo.trainset.to_raw_iid(item)
    print("物品{}和物品{}的相似度为：{}".format(row_item_id, row_like_item_id, algo.sim[inner_item_id][item]))

全部数据中最相似的k个商品(内部id):[35, 240, 258, 284, 320, 333, 404, 438, 457, 606]
全部数据中最相似的k个商品(外部id):['320', '964', '1038', '1066', '991', '374', '344', '1176', '670', '835']

物品1和物品320的相似度为：0.1800205749557739
物品1和物品964的相似度为：0.15724272550828775
物品1和物品1038的相似度为：0.610170215847752
物品1和物品1066的相似度为：0.408248290463863
物品1和物品991的相似度为：-0.22973414586817037
物品1和物品374的相似度为：0.5107539184552492
物品1和物品344的相似度为：0.16205747826813258
物品1和物品1176的相似度为：-0.581675050747111
物品1和物品670的相似度为：0.0691086488735382
物品1和物品835的相似度为：0.0


## ItemCF变种

1. 构建当前用户u的物品喜好列表<br/>
2. 对于喜好列表中的所有物品i，分别根据计算出来的物品之间的相似度矩阵获取当前商品i的相似物品<br/>
3. 思考：可以认为物品i和它的相似物品是比较类似；同时我们可以认为如果此时我们将相似物品推荐给用户u其实是可以的。但是此时用户u的喜好商品是多个(列表)，所以产生的推荐列表不能直接拿某一个物品i的相似物品进行推荐<br/>
4. 根据用户u对喜好列表中的物品的偏好程度，更新对应相似物品的权重值<br/>
5. 计算所有相似物品的加权平均相似度<br/>
6. 获取相似度最大的前N个物品作为推荐列表<br/>



In [11]:
# 输出相似度矩阵
print("物品相似度矩阵：")
print(algo.sim)
pro_2_pro_sim_path = "./datas/pro_2_pro_sim.txt"
with open(pro_2_pro_sim_path, 'w', encoding='UTF-8') as writer:
    n = algo.sim.shape[0]
    for i in range(n):
        for j in range(n):
            # 转换成为外部实际的物品id
            row_i = algo.trainset.to_raw_iid(i)
            row_j = algo.trainset.to_raw_iid(j)
            # 获取两个物品之间的相似度
            sim = algo.sim[i][j]
            # 输出
            writer.writelines("{},{},{}\n".format(row_i, row_j, sim))

物品相似度矩阵：
[[1.         0.2070249  0.36994668 ... 0.         0.         0.        ]
 [0.2070249  1.         0.08002346 ... 0.         0.         0.        ]
 [0.36994668 0.08002346 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


## 输入的用户-物品评分的构建
由显示评分和隐式评分组成，一般情况下，我们都需要根据定义的规则对用户行为数据进行处理，从而计算出隐式评分


## 冷启动

冷启动问题指的其实就是推荐系统如何给新增用户推荐物品列表，以及当一个新物品上架后，如何将该物品推荐给用户??

用户的冷启动：<br/>
1. 利用TopN的热门商品作为推荐列表<br/>
2. 当用户只要访问一个物品/对一个物品产生偏好信息后，我们就可以基于ItemCF为该用户产生推荐列表<br/>
3. 收集一些用户的信息然后基于用户特征信息构建用户的相似度矩阵，从而可以实现推荐(不是UserCF)<br/>


物品的冷启动：<br/>
1. 利用物品的特征属性，基于物品特征属性构建物品之间的相似度矩阵；比如将新增加的物品和老的物品数据进行聚类操作，当同一个簇中的老物品被访问的时候，就可以直接获取新的物品作为推荐列表(权重的形式)<br/>
2. 当一个新物品被用户产生偏好信息后，我们就可以将这个新物品推荐给和这个用户相似的其它用户


# <br/>

# <br/>

# 强调

应用最广泛的其实还是协同过滤算法；在协同过滤算法中，最常见的两种算法分别是：基于用户的协同过滤(UserCF)和基于物品的系统过滤(ItemCF)

### UserCF

核心思想：对于当前用户u而言，根据用户u和其它用户的历史行为数据/用户-物品评分信息，找出和用户u最相似的K个近邻用户，然后将这K个近邻用户喜好的物品推荐给用户u。

用户相似度度量方式：本质的思想其实都是一样的；对于用户u和用户v而言，用户u和用户v之间的相似度其实是受用户u和用户v共同评价过的商品分数影响的。

“找出和用户u最相似的K个近邻用户，然后将这K个近邻用户喜好的物品推荐给用户u”这个过程其实是反过来计算的；<br/>
1. 用户u对于未产生行为物品i的喜好程度/评分的计算过程：获取所有评论过物品i的其它用户，然后再这些其它用户中计算和当前用户u最相似的K个用户，然后使用这K个用户对物品i的评分计算得到用户u对于物品i的评分（评分计算公式：加权平均求和、基于均值计算的加权平均求和、基于偏置项计算的加权平均求和） ===> 利用的是相似用户喜好的物品是类似的这种意思
2. 迭代第一步，计算出用户u对于所有物品的喜好程度/评分。
3. 获取用户u对于所有物品评分中最高的N个评分物品作为最终的推荐列表

### ItemCF

核心思想：对于当前用户u而言，根据用户u历史的偏好信息/偏好的物品列表，找出和列表中物品最相似的K个近邻物品，然后将这K个近邻物品推荐给用户u。

相对于UserCF而言，唯一的区别在于：在ItemCF中计算的是物品之间的相似度，推荐的原理是基于用户历史喜好什么物品，那么未来可能会喜好和这个物品类似的其它物品，从而将相似物品作为推荐列表

物品相似度度量方式：本质的思想其实都是一样的；对于物品i和物品j而言，物品i和物品j之间的相似度其实是受物品i和物品j被共同评价过的用户评分数影响的。

“根据用户u历史的偏好信息/偏好的物品列表，找出和列表中物品最相似的K个近邻物品，然后将这K个近邻物品推荐给用户u”这个过程其实是反过来计算的；<br/>
1. 根据用户u历史的偏好信息/偏好的物品列表，对物品列表中的每个物品做一下操作；
2. 用户u对于未产生行为物品i的喜好程度/评分的计算过程：获取用户u评估过的其它物品j；然后计算这些物品与物品i之间的相似度，获取和物品i最相似的K个物品；然后使用用户u对于这K个物品的评分计算用户u对于物品i的评分。（评分计算公式：加权平均求和、基于均值计算的加权平均求和、基于偏置项计算的加权平均求和） ===> 利用：如果物品i和物品j比较相似，并且用户u喜好物品j的情况下，可以认为用户u也会喜好物品i
4. 迭代第二步，计算出用户u对于所有物品的喜好程度/评分。
5. 获取用户u对于所有物品评分中最高的N个评分物品作为最终的推荐列表

### 评分矩阵的初始化

用户id和物品id一般我们可以直接用数据库中获取即可；但是对于评分这个信息，一般情况下，比较可靠的一种方式是通过问卷调查或者用户的主动评分 -> 显示评分；但是一般情况下，显示评分数据都比较少，所以我们一般需要通过其它方式来主动的获取评分数据，这种方式得到的评分叫做隐式评分，一般隐式评分的得到是需要根据业务规则对用户行为信息做一个量化的转换，eg: 单曲循环=5, 分享=4, 收藏=3, 主动播放=2 , 听完=1, 跳过=-1 , 拉黑=-5

### 冷启动问题

冷启动问题指的其实就是推荐系统如何给新增用户推荐物品列表，以及当一个新物品上架后，如何将该物品推荐给用户??

用户的冷启动：<br/>
1. 利用TopN的热门商品作为推荐列表<br/>
2. 当用户只要访问一个物品/对一个物品产生偏好信息后，我们就可以基于ItemCF为该用户产生推荐列表<br/>
3. 收集一些用户的信息然后基于用户特征信息构建用户的相似度矩阵，从而可以实现推荐(不是UserCF)<br/>


物品的冷启动：<br/>
1. 利用物品的特征属性，基于物品特征属性构建物品之间的相似度矩阵；比如将新增加的物品和老的物品数据进行聚类操作，当同一个簇中的老物品被访问的时候，就可以直接获取新的物品作为推荐列表(权重的形式)<br/>
2. 当一个新物品被用户产生偏好信息后，我们就可以将这个新物品推荐给和这个用户相似的其它用户
